In [166]:
# Create a Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [167]:
# Create a Pandas DataFrame with 10 random numbers

import numpy as np
import pandas as pd
random_numbers_df = pd.DataFrame(np.random.randint(0,50,size=(5, 2)), columns=list('AB'))

In [168]:
# Convert the Pandas DataFrame to a Spark DataFrame

spark_random_numbers_df = spark.createDataFrame(random_numbers_df)

/usr/local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [169]:
# How many tables exist in the SparkSession?
spark.catalog.listTables()

[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [170]:
# Create a temporary view of the dataframe called 'temp'
spark_random_numbers_df.createOrReplaceTempView('temp')

In [171]:
# How many tables exist in the SparkSession now?

spark.catalog.listTables()

[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [172]:
# Use Spark to read in airports.csv with headers

spark_airports = spark.read.csv('airports.csv', header= True)

In [173]:
# What type is the object that holds the csv you read in?
type(spark_airports)

pyspark.sql.dataframe.DataFrame

In [174]:
# Use Spark to read in flights_small.csv with headers
spark_flight_small = spark.read.csv('flights_small.csv', header= True)
spark_flight_small

DataFrame[year: string, month: string, day: string, dep_time: string, dep_delay: string, arr_time: string, arr_delay: string, carrier: string, tailnum: string, flight: string, origin: string, dest: string, air_time: string, distance: string, hour: string, minute: string]

In [175]:
# Add a column to flights called 'duration_hrs'

spark_flight_small = spark_flight_small.withColumn('duration_hrs', spark_flight_small['air_time']/60)

In [176]:
# Filter flights for all flights further than 1000 miles
flights_further_than_1000 = spark_flight_small.filter('distance > 1000')

In [177]:
flights_further_than_1000.show(1)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|         6.0|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
only showing top 1 row



In [178]:
# Create a new dataset called 'columns_1' with tail numbers, origin, and destination
columns_1 = spark_flight_small.select('tailnum', 'origin', 'dest')

In [179]:
columns_1.show(5)

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N846VA|   SEA| LAX|
| N559AS|   SEA| HNL|
| N847VA|   SEA| SFO|
| N360SW|   PDX| SJC|
| N612AS|   SEA| BUR|
+-------+------+----+
only showing top 5 rows



In [180]:
# Create a filter for flights starting from 'SEA'
# On columns_1, create a filter for flights going to 'PDX'
filter1 = columns_1.origin == 'SEA'
filter2 = columns_1.dest == 'PDX'

In [181]:
# Create a new dataset only with flights going from SEA to PDX
flights_SEA_to_PDX = columns_1.filter(filter1)
flights_SEA_to_PDX = columns_1.filter(filter2)

In [182]:
# Calculate the average speed of all flights and call it 'avg_speed
spark_flight_small = spark_flight_small.withColumn('avg_speed', spark_flight_small['distance']/spark_flight_small['duration_hrs'])

In [183]:
# What's the type of avg_speed?

type(spark_flight_small['avg_speed'])

pyspark.sql.column.Column

In [184]:
# Create a new dataset with each flight's tailnum, origin, destination and full date

small_flights = spark_flight_small.selectExpr('tailnum', 'origin', 'dest','concat_ws("-",year, month, day) as date')

In [185]:
small_flights.show(5)

+-------+------+----+---------+
|tailnum|origin|dest|     date|
+-------+------+----+---------+
| N846VA|   SEA| LAX|2014-12-8|
| N559AS|   SEA| HNL|2014-1-22|
| N847VA|   SEA| SFO| 2014-3-9|
| N360SW|   PDX| SJC| 2014-4-9|
| N612AS|   SEA| BUR| 2014-3-9|
+-------+------+----+---------+
only showing top 5 rows



In [186]:
# In flights, convert the distance column to be type float
spark_flight_small = spark_flight_small.withColumn('distance',spark_flight_small['distance'].cast("float").alias('distance'))

In [187]:
# In flights, convert the air_time column to be type float
spark_flight_small = spark_flight_small.withColumn('air_time',spark_flight_small['air_time'].cast("float").alias('air_time'))

In [188]:
# What's the shortest flight time?
spark_flight_small.describe('air_time').show()

+-------+------------------+
|summary|          air_time|
+-------+------------------+
|  count|              9925|
|   mean|152.88423173803525|
| stddev|  72.8656286392139|
|    min|              20.0|
|    max|             409.0|
+-------+------------------+



In [189]:
# What's the longest flight length?
spark_flight_small.describe('distance').show()

+-------+-----------------+
|summary|         distance|
+-------+-----------------+
|  count|            10000|
|   mean|        1208.1516|
| stddev|656.8599023464376|
|    min|             93.0|
|    max|           2724.0|
+-------+-----------------+



In [ ]:
# Filter the flights dataset for flights that left from PDX